# Using TensoFlow low level API

## Resources

- https://www.tensorflow.org/guide/low_level_intro
- https://medium.com/ymedialabs-innovation/how-to-use-dataset-and-iterators-in-tensorflow-with-code-samples-3bb98b6b74ab
- https://www.tensorflow.org/guide/datasets
- https://www.tensorflow.org/guide/graphs

In [1]:
import tensorflow as tf
from pprint import pprint

TensorFlow has two main components:
- Graphs (`tf.Graph`): the computational graph to be run.
- Sessions (`tf.Session`): the runtime that runs the graph.

## Graphs

Graphs comprise two main units:
- Operations (`tf.Operation`) ("ops"): the nodes of the graph, representing calculations on tensors.
- Tensors (`tf.Tensor`): the edges of the graph, representing the values that flow through the graph.

Operations are really blueprints for functions that get run at runtime (in a session): tensors are not inputs, but rather passed to the op's constructor. The oputput is then produced at runtime.

Tensors are blueprints as well: they don't hold values outside a session.

If we print a tensor or an operation we get information on the kind of objects that they represent in the graph.

### TensorBoard

Graphs can be visualised in TensorBoard (navigate to the directory where the event file is saved and run `tensorboard --logdir .`). Notice that every time a cell defining a graph is run a new graph is added to the default graphs (`tf.get_default_graph`) (which is also reflected in the change of the names of the tensors and ops). TensorBoard is served as a web app on `localhost:6006`.

To reset the default graph: `tf.reset_default_graph()`.

In [2]:
# Cleans the default graph of all the operations associated
# to it.
tf.reset_default_graph()

# Ops
# Tensors
t1 = tf.constant([[1., 2.], [4., 4.]])
t2 = tf.constant([[2., 2.], [2., 2.]])

# Adds the tensor together
tensor_sum = tf.add(t1, t2)
tensor_mult = tf.matmul(tensor_sum, tf.constant([[3., 0.], [0., 3.]]))

# Write to TensorBoard: creates an events file in the current
# directory.
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())
writer.flush()

print(t1)
print(t2)
print(tensor_sum)
print(tensor_mult)

print("\nOperations in the default graph:")
pprint(tf.get_default_graph().get_operations())

Tensor("Const:0", shape=(2, 2), dtype=float32)
Tensor("Const_1:0", shape=(2, 2), dtype=float32)
Tensor("Add:0", shape=(2, 2), dtype=float32)
Tensor("MatMul:0", shape=(2, 2), dtype=float32)

Operations in the default graph:
[<tf.Operation 'Const' type=Const>,
 <tf.Operation 'Const_1' type=Const>,
 <tf.Operation 'Add' type=Add>,
 <tf.Operation 'Const_2' type=Const>,
 <tf.Operation 'MatMul' type=MatMul>]


## Sessions

Sessions can run operations to produce tensors. Unless otherwise specified when it's instantiated, a session is associated to the default graph.

In [3]:
sess = tf.Session()

# Evaluate tensors
print("Value of t1:")
print(sess.run(t1))
print("Value of t2:")
print(sess.run(t2), "\n")

# Run operations
print("Output of tensor_sum:")
print(sess.run(tensor_sum))
print("Output of tensor_mult:")
print(sess.run(tensor_mult), "\n")

# Evaluate more than one tensor/operations
# with a single call to Session.run()
print("Multiple evaluations:")
print(sess.run({
    'tensor_sum': tensor_sum,
    'tensor_mult': tensor_mult
}))

Value of t1:
[[1. 2.]
 [4. 4.]]
Value of t2:
[[2. 2.]
 [2. 2.]] 

Output of tensor_sum:
[[3. 4.]
 [6. 6.]]
Output of tensor_mult:
[[ 9. 12.]
 [18. 18.]] 

Multiple evaluations:
{'tensor_sum': array([[3., 4.],
       [6., 6.]], dtype=float32), 'tensor_mult': array([[ 9., 12.],
       [18., 18.]], dtype=float32)}


Operations can also be evaluated with their `eval()` method by passing a session to its `session` keyword argument.

In [4]:
print("Value of t1:")
print(t1.eval(session=sess), "\n")

print("Output of tensor_mult:")
print(tensor_mult.eval(session=sess))

Value of t1:
[[1. 2.]
 [4. 4.]] 

Output of tensor_mult:
[[ 9. 12.]
 [18. 18.]]


Finally, we can define session object using a context.

In [5]:
with tf.Session() as temp_sess:
    print("Value of t1:")
    print(t1.eval(session=temp_sess), "\n")
    
    print("Output of tensor_mult:")
    print(temp_sess.run(tensor_mult))

Value of t1:
[[1. 2.]
 [4. 4.]] 

Output of tensor_mult:
[[ 9. 12.]
 [18. 18.]]


A within a single call to `tf.Session.run()` a tensor has a single value. This doesn't happen across multiple calls of `run()` and can be seen e.g. when generating random tensors: within the same run the generated value is consistent, while it changes between different runs.

In [6]:
random_tensor = tf.random_uniform(shape=(2,))

random_plus_one = random_tensor + 1.
random_plus_two = random_tensor + 2.

print("First call to .run(): ", sess.run(random_tensor))
print("Second call to .run(): ", sess.run(random_tensor))

# In the following, the two new tensors read the same value for
# the input tensor.
print(
    "Calling run on two operations with random_tensor as the input:")
print(sess.run(
        {
            "plus_one": random_plus_one,
            "plus_two": random_plus_two
        }
    )
)

First call to .run():  [0.32275498 0.54546475]
Second call to .run():  [0.08892083 0.85178363]
Calling run on two operations with random_tensor as the input:
{'plus_one': array([1.0299999, 1.2709372], dtype=float32), 'plus_two': array([2.0299997, 2.2709372], dtype=float32)}


## Placeholders

Operations can be passed external input values, which are indicated by `placeholder`s at the moment of graph creation. When we have a session execute the graph, we need to give values to the placeholders, which we do via the `feed_dict` keyword argument of the `Session.run()` method.

In [7]:
tf.reset_default_graph()

t1 = tf.placeholder(shape=(2, 2), dtype=tf.float32)
t2 = tf.placeholder(shape=(2, 2), dtype=tf.float32)

matmul_t1_t2 = tf.matmul(t1, t2)

matmul_t1_t2

<tf.Tensor 'MatMul:0' shape=(2, 2) dtype=float32>

The session `sess` was associated to a previous version of the default graph, which we erased with the above call to `tf.reset_default_graph()`, therefore if we want to use a session with the same name it has to be re-instantiated so it is associated with the current version of the graph.

As an alternative, we can re-create it using a context.

In [8]:
sess = tf.Session()

sess.run(
    matmul_t1_t2,
    feed_dict={
        t1: [[1, 0], [0, 2]],
        t2: [[1, 2], [3, 4]]
    }
)

array([[1., 2.],
       [6., 8.]], dtype=float32)

We don't need to specify a tensor's shape in the placeholder: dimensions can be inferred from what is passed to `feed_dict` (and errors will be thrown is there are inconsistencies).

In [9]:
tf.reset_default_graph()

t1 = tf.placeholder(dtype=tf.float32)
t2 = tf.placeholder(dtype=tf.float32)

matmul_t1_t2 = tf.matmul(t1, t2)

with tf.Session() as sess:
    print(sess.run(
        matmul_t1_t2,
        feed_dict = {
            t1: [[1, 2, 3], [1, 1, 1], [5, 7, 5]],
            t2: [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
        }
    ))

[[1. 2. 3.]
 [1. 1. 1.]
 [5. 7. 5.]]


## Datasets

`tf.data` allows for inputting data into a graph in a more standardised and automated way than a placeholder. Given a dataset, we can create an iterator and use it to retrieve each datapoint, giving a value to a tensor. At every call to `Session.run()`, the iterator will return the next datapoint and when no more datapoints are present it raises a `tf.errors.OutOfRangeError` exception.

In [5]:
tf.reset_default_graph()
sess = tf.Session()

data = [
    [0, 1],
    [2, 4],
    [9, 4],
    [5, 8]
]

dataset = tf.data.Dataset.from_tensor_slices(data)
iterator = dataset.make_one_shot_iterator() # This will give a deprecation warning.
next_element = iterator.get_next()

while True:
    try:
        print(sess.run(next_element))
    except tf.errors.OutOfRangeError:
        break

[0 1]
[2 4]
[9 4]
[5 8]


In [6]:
tf.reset_default_graph()
sess = tf.Session()

data = [
    [0, 1],
    [2, 4],
    [9, 4],
    [5, 8]
]

dataset = tf.data.Dataset.from_tensor_slices(data)
iterator = tf.compat.v1.data.make_one_shot_iterator(dataset)
next_element = iterator.get_next()

while True:
    try:
        print(sess.run(next_element))
    except tf.errors.OutOfRangeError:
        break

[0 1]
[2 4]
[9 4]
[5 8]


## Building layers

Let's create a simple linear model using a fully connected layer.

In [37]:
tf.reset_default_graph()

In [38]:
sess_model = tf.Session()

Generate data.

In [39]:
data = tf.random_uniform(shape=[50, 1], dtype=tf.float32)

data_eval = sess_model.run(data)

data_eval[:10]

array([[0.18897486],
       [0.15182555],
       [0.75612843],
       [0.5720967 ],
       [0.43041754],
       [0.49657822],
       [0.24570096],
       [0.55756056],
       [0.9580214 ],
       [0.46783197]], dtype=float32)

Create a model, a single-layer fully-connected neural network.

In [40]:
x = tf.placeholder(tf.float32, shape=[None,1])

model = tf.layers.Dense(units=1, dtype=tf.float32)

y = model(x)

The Dense layer contains weights that must be initialised to random values. To do so, instantiate and run a global variable initializer.

In [51]:
init = tf.global_variables_initializer()

sess_model.run(init)

Evaluate the model with randomly initialised weights on the whole dataset.

In [52]:
sess_model.run(
    y,
    feed_dict={x: data_eval}
)

array([[0.04625671],
       [0.03716341],
       [0.18508291],
       [0.14003617],
       [0.10535635],
       [0.12155097],
       [0.06014196],
       [0.13647805],
       [0.23450169],
       [0.11451454],
       [0.24171309],
       [0.16552752],
       [0.01935052],
       [0.2287936 ],
       [0.24144986],
       [0.12410813],
       [0.23869364],
       [0.2079743 ],
       [0.0524101 ],
       [0.03536046],
       [0.02092561],
       [0.12367263],
       [0.0319729 ],
       [0.0755981 ],
       [0.24131234],
       [0.1751338 ],
       [0.14065267],
       [0.225483  ],
       [0.23638101],
       [0.12443634],
       [0.00733379],
       [0.06231842],
       [0.18902552],
       [0.04404125],
       [0.1812489 ],
       [0.09022603],
       [0.11428128],
       [0.03689266],
       [0.00172341],
       [0.1406048 ],
       [0.09875742],
       [0.06419674],
       [0.14924522],
       [0.20968202],
       [0.1521935 ],
       [0.13257574],
       [0.2117897 ],
       [0.036